# 🧠 Alzheimer MRI Sınıflandırması: Keşifçi Veri Analizi (EDA)

Bu notebook, model eğitimine başlamadan önce veri setimizin yapısını, içeriğini ve karşılaşılabilecek potansiyel problemleri anlamak için hazırlanmıştır.

**Amaçlar:**
- Veri dağılımını anlamak
- Görselleri tanımak
- Augmentasyon ihtiyacını görmek
- Model için gerekli stratejilerin zeminini oluşturmak


In [ ]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.dataset import AlzheimerDataset, _list_classes
from src.transforms import base_train_transform, default_transform, IMAGENET_MEAN, IMAGENET_STD

from torchvision import transforms

In [ ]:
data_root = "../data/processed"
train_path = os.path.join(data_root, "train")
classes = _list_classes(train_path)

dataset = AlzheimerDataset(train_path, transform=default_transform)

print("Sınıflar:", classes)
print("Toplam örnek sayısı:", len(dataset))

In [ ]:
counts = {cls: dataset.labels.count(i) for i, cls in enumerate(classes)}
df = pd.DataFrame(list(counts.items()), columns=["Class", "Count"])
df["Percentage"] = df["Count"] / df["Count"].sum() * 100

plt.figure(figsize=(10,6))
sns.barplot(x="Class", y="Count", data=df, palette="viridis")
plt.title("Sınıf Dağılımı", fontsize=16)
plt.show()

df

In [ ]:
sample_transform = transforms.Compose([
    transforms.Resize((176,208)),
    transforms.ToTensor()
])

eda_dataset = AlzheimerDataset(train_path, transform=sample_transform)

fig, axes = plt.subplots(1, 4, figsize=(12,4))
for i in range(4):
    img, label = eda_dataset[i]
    axes[i].imshow(img.permute(1,2,0))
    axes[i].set_title(classes[label])
    axes[i].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
sample_img, label = eda_dataset[0]
augmented_img = base_train_transform(transforms.ToPILImage()(sample_img))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4))
ax1.imshow(sample_img.permute(1,2,0))
ax1.set_title("Orijinal")
ax1.axis("off")

ax2.imshow(augmented_img.permute(1,2,0))
ax2.set_title("Augmented (base_train_transform)")
ax2.axis("off")
plt.show()

## 📌 Sonuç ve Çıkarımlar

1. **Sınıf Dengesizliği:** `ModerateDemented` sınıfı çok az sayıda örneğe sahip.
2. **Ön İşleme İhtiyacı:** Görseller standart boyuta (224x224) resize edilmeli ve normalize edilmelidir.
3. **Augmentasyon İhtiyacı:** Özellikle `VeryMildDemented` ve `ModerateDemented` sınıflarda gerekli.
4. **Model Gereksinimleri:** WeightedRandomSampler + Focal Loss gibi yöntemler, bu dengesizlikleri aşmak için kritik öneme sahiptir.
